In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import tqdm as tqdm
import shapely
import cdsapi
import os
import glob
import pickle

import matplotlib.pyplot as plt
from geographiclib.geodesic import Geodesic
import vptree

import gtc_functions

In [3]:
grib_root = '/Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/EFs/weather_data/ecmwf/'

event_names = ['FLORENCE', 'HARVEY', 'MATTHEW', 'MICHAEL']
xa_dict = {}

# assign xarrays to labelled dictionary
for name in tqdm.tqdm(event_names,total=len(event_names)):
	grib_path = gtc_functions.get_path(grib_root + '.'.join((name,'grib')))
	xa_dict[name] = xr.load_dataset(grib_path, engine="cfgrib")

100%|██████████| 4/4 [00:19<00:00,  4.78s/it]


In [4]:
# TODO: better common path solution. Lisanne's use of Owen's functions? Cambridge-hosted data server?
google_drive_personal_key = '/Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/'

# xbd observation points
df_xbd_points_path = google_drive_personal_key + 'datasets/xBD_data/xbd_points_posthurr_reformatted.pkl'
df_xbd_points = pd.read_pickle(df_xbd_points_path)
df_xbd_points.head()

,geometry,damage_class,disaster_name,capture_date,lon,lat
0,POINT (-77.92443 34.78850),2,FLORENCE,2018-09-20 16:04:41+00:00,-77.924432,34.788502
1,POINT (-77.92459 34.78817),1,FLORENCE,2018-09-20 16:04:41+00:00,-77.924586,34.788174
2,POINT (-85.61007 30.20004),0,MICHAEL,2018-10-13 16:48:15+00:00,-85.610074,30.200042
3,POINT (-85.61057 30.20001),0,MICHAEL,2018-10-13 16:48:15+00:00,-85.610569,30.200012
4,POINT (-85.61055 30.20060),1,MICHAEL,2018-10-13 16:48:15+00:00,-85.610547,30.200601


In [5]:
xa_dict.keys()

dict_keys(['FLORENCE', 'HARVEY', 'MATTHEW', 'MICHAEL'])

In [6]:
xa_dict['MATTHEW']

<xarray.Dataset>
Dimensions:     (time: 7, step: 24, latitude: 121, longitude: 101)
Coordinates:
    number      int64 0
  * time        (time) datetime64[ns] 2016-09-30 2016-10-01 ... 2016-10-06
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 24.0 23.9 23.8 23.7 ... 12.3 12.2 12.1 12.0
  * longitude   (longitude) float64 -80.0 -79.9 -79.8 ... -70.2 -70.1 -70.0
    valid_time  (time, step) datetime64[ns] 2016-09-30T01:00:00 ... 2016-10-07
Data variables:
    u10         (time, step, latitude, longitude) float32 nan nan ... nan nan
    v10         (time, step, latitude, longitude) float32 nan nan ... nan nan
    d2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
    t2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
    sp          (time, step, latitude, longitude) float32 nan nan ... nan nan
    tp          (time, step, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-03-12T22:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
df_xbd_points_grouped = df_xbd_points.groupby('disaster_name')

weather_headers = [str(el) for el in list(list((xa_dict.values()))[0].data_vars)]

for event_name, group in df_xbd_points_grouped:
	print(event_name)
	print(xa_dict[event_name])

FLORENCE
<xarray.Dataset>
Dimensions:     (time: 6, step: 24, latitude: 401, longitude: 601)
Coordinates:
    number      int64 0
  * time        (time) datetime64[ns] 2018-09-12 2018-09-13 ... 2018-09-17
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 40.0 39.9 39.8 39.7 39.6 ... 0.3 0.2 0.1 0.0
  * longitude   (longitude) float64 -110.0 -109.9 -109.8 ... -50.2 -50.1 -50.0
    valid_time  (time, step) datetime64[ns] 2018-09-12T01:00:00 ... 2018-09-18
Data variables:
    u10         (time, step, latitude, longitude) float32 nan nan ... nan nan
    v10         (time, step, latitude, longitude) float32 nan nan ... nan nan
    d2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
    t2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
    sp          (time, step, latitude, longitude) float32 nan nan ... nan nan
    tp          (time, step, latitude, longitu

In [8]:
def restrict_coords(
	av_lon: float,
	av_lat: float,
	weather_lons: list[float],
	weather_lats: list[float],
	buffer: float
	) -> tuple[list]:
	"""TODO: docstring"""
	min_lon, max_lon = av_lon-buffer, av_lon+buffer
	min_lat, max_lat = av_lat-buffer, av_lat+buffer

	rest_weather_lons = [num for num in weather_lons if num >= min_lon and num <= max_lon]
	rest_weather_lats = [num for num in weather_lats if num >= min_lat and num <= max_lat]
	
	return rest_weather_lons, rest_weather_lats


def geoddist(p1, p2):
	# lat1, lon1, lat2, lon2
    return Geodesic.WGS84.Inverse(p1[1], p1[0], p2[1], p2[0])['s12']

In [9]:
df_xbd_points_grouped = df_xbd_points.groupby('disaster_name')
# TODO: update with more values (from API calling)
# TODO: make into a function with adjustable range (post-api calling). This makes it the calculation side, whereas api
# calling is for additionoal visualisation
parameters = ['u10','v10','d2m','t2m','sp','tp']
dictionary_list = []

for event_name, group in df_xbd_points_grouped:
    xa = xa_dict[event_name]
    df = xa_dict[event_name].to_dataframe()

    # remove missing values (i.e. lat-lons in ocean)
    df_nonans = df.dropna(how='any')
    # assign latitude longitude multiindex to columns
    df_flat = df_nonans.reset_index()

    # coarse df spatial limitation for faster standardisation
    av_lon, av_lat = group.lon.mean(), group.lat.mean()
    df_flat = gtc_functions.limit_df_spatial_range(df_flat, [av_lat,av_lon], distance_buffer=2)
    df_flat = gtc_functions.standardise_dfs(df_flat)

    # iterate through each row in the xbd event df
    for i,row in tqdm.tqdm(group.iterrows(), total=len(group)):
        poi = shapely.geometry.point.Point(row.lon, row.lat)
        # further restrict df for specific point
        df_flat_specific = gtc_functions.limit_df_spatial_range(df_flat, [row.lat,row.lon], min_number=1)
        closest_ind = gtc_functions.find_index_closest_point_in_col(poi, df_flat_specific, 'geometry')
        # need to find lon, lat of closest point rather than the index
        # TODO: not sure that max is most relevant for all – parameterisation
        df_maxes = df_flat_specific[df_flat_specific['geometry'] == df_flat_specific.loc[closest_ind]['geometry']]
        maxs = df_maxes[parameters].abs().max()
            
        # generate dictionary of weather values
        dict_data = {k: maxs[k] for k in weather_headers}
        dict_data['xbd_index'] = row.name
        dict_data['name'] = event_name

        dictionary_list.append(dict_data)

weather_params_df = pd.DataFrame.from_dict(dictionary_list)

100%|██████████| 7158/7158 [02:21<00:00, 50.72it/s]


In [11]:
for n in weather_params_df['name'].unique()[:]:
	print(n)
	print(weather_params_df[weather_params_df['name'] == n].isna().sum())

FLORENCE
u10          0
v10          0
d2m          0
t2m          0
sp           0
tp           0
xbd_index    0
name         0
dtype: int64
HARVEY
u10          0
v10          0
d2m          0
t2m          0
sp           0
tp           0
xbd_index    0
name         0
dtype: int64
MATTHEW
u10          0
v10          0
d2m          0
t2m          0
sp           0
tp           0
xbd_index    0
name         0
dtype: int64
MICHAEL
u10          0
v10          0
d2m          0
t2m          0
sp           0
tp           0
xbd_index    0
name         0
dtype: int64


## API Working

In [13]:
# fetch relevant weather files for api calls

### LOAD IN RELEVANT DFS
# /Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/xBD_data/xbd_points_posthurr_reformatted.pkl
# xbd observation points
df_xbd_points_path = google_drive_personal_key + '/datasets/processed_data/metadata_pickle/lnglat_pre_pol_post_damage.pkl'
df_xbd_points = pd.read_pickle(df_xbd_points_path)
# xbd noaa 6-hourly
df_xbd_noaa_pkl_path = google_drive_personal_key + 'datasets/EFs/weather_data/noaa_xbd_hurricanes.pkl'
df_xbd_noaa_weather = pd.read_pickle(df_xbd_noaa_pkl_path)

In [14]:
df_xbd_points = gtc_functions.standardise_xbd_obs_df(df_xbd_points)
df_xbd_points.head()

,geometry,polygon_lnglat,point_xy,polygon_xy,disaster_name,image_name,capture_date,match_num,damage_class,lon,lat
0,POINT (-79.04236 33.58624),POLYGON ((-79.04246877521777 33.58619656219695...,POINT (21.859657142647336 11.540384007147074),"POLYGON ((0 23.07542987740177, 5.4400605824964...",FLORENCE,hurricane-florence_00000006_pre_disaster.png,2018-08-23 16:38:58+00:00,0,0,-79.042360,33.586243
1,POINT (-79.04067 33.58624),POLYGON ((-79.04076716312134 33.58626511642874...,POINT (364.29112858833366 6.050993941743565),"POLYGON ((343.8285603480572 0, 343.67546136437...",FLORENCE,hurricane-florence_00000006_pre_disaster.png,2018-08-23 16:38:58+00:00,1,0,-79.040667,33.586238
2,POINT (-79.03924 33.58618),POLYGON ((-79.03933584242897 33.58624243839649...,POINT (652.7525494937349 15.915749218488404),"POLYGON ((633.4092574824836 0, 636.30875820509...",FLORENCE,hurricane-florence_00000006_pre_disaster.png,2018-08-23 16:38:58+00:00,2,0,-79.039242,33.586175
3,POINT (-79.03905 33.58622),POLYGON ((-79.03914144408215 33.58623935704581...,POINT (692.2940274077046 4.47976753618199),"POLYGON ((672.7393984390319 0, 680.08814965569...",FLORENCE,hurricane-florence_00000006_pre_disaster.png,2018-08-23 16:38:58+00:00,3,0,-79.039045,33.586219
4,POINT (-79.03744 33.58610),POLYGON ((-79.03740598016368 33.58617995558616...,POINT (1017.8013436211089 27.135715154384336),"POLYGON ((1024 7.703541722778522, 1013.8706024...",FLORENCE,hurricane-florence_00000006_pre_disaster.png,2018-08-23 16:38:58+00:00,4,0,-79.037438,33.586100


In [15]:
event_api_info, start_end_dates, areas = gtc_functions.return_relevant_event_info(
	df_xbd_points, 
	df_xbd_noaa_weather, 
	distance_buffer=6,
	verbose=True)

event_name: FLORENCE
min_date: 2018-09-14 06:00:00, max_date: 2018-09-15 12:00:00, event_area: [41.0, -84.0, 28.0, -73.0]
event_name: HARVEY
min_date: 2017-08-26 06:00:00, max_date: 2017-08-29 18:00:00, event_area: [36.0, -101.0, 23.0, -90.0]
event_name: MATTHEW
min_date: 2016-10-04 00:00:00, max_date: 2016-10-05 06:00:00, event_area: [25.0, -79.0, 12.0, -68.0]
event_name: MICHAEL
min_date: 2018-10-10 00:00:00, max_date: 2018-10-11 06:00:00, event_area: [37.0, -91.0, 24.0, -80.0]


In [20]:
def generate_api_dict(
	weather_params: list[str],
	time_info_dict: dict,
	area: list[float],
	format: str
) -> dict:
	"""Generate api dictionary format for single month of event"""

	api_call_dict = {
		"variable": weather_params,
		"area": area,
		"format": format
	} | time_info_dict

	return api_call_dict


def return_parameter_strings(
	dict_keys: list[str]
):
	"""Look up weather parameters in a dictionary so they can be entered as short strings rather than typed out in full.
	Key:value pairs ordered in expected importance
	TODO: may change keys to acronyms
	"""

	weather_dict = {
		't0': '2m_dewpoint_temperature', 't1': '2m_temperature', 't2': 'skin_temperature',
		'tp': 'total_precipitation',
		'sp': 'surface_pressure',
		's0': 'skin_reservoir_content', 's1': 'volumetric_soil_water_layer_1', 's2': 'volumetric_soil_water_layer_2', 
		's3': 'volumetric_soil_water_layer_3', 's4': 'volumetric_soil_water_layer_4',
		'h0': 'surface_latent_heat_flux', 'h1': 'surface_sensible_heat_flux', 
		'r0': 'surface_net_solar_radiation', 'r1': 'surface_net_thermal_radiation', 'r2': 'surface_solar_radiation_downwards',
		'r3': 'surface_thermal_radiation_downwards',
		'e0': 'total_evaporation', 'e1': 'evaporation_from_bare_soil', 'e2': 'potential_evaporation',
		'run0': 'runoff', 'run1': 'sub-surface_runoff','run2': 'surface_runoff',
		'w0': '10m_u_component_of_wind', 'w1': '10m_v_component_of_wind', 
	}

	weather_params = []
	for key in dict_keys:
		weather_params.append(weather_dict.get(key))
	
	return weather_params


def generate_times_from_start_end(
	start_end_dates: list[tuple[pd.Timestamp]]
) -> dict:
	"""Generate dictionary containing ecmwf time values.
	
	TODO: update so can span multiple months accurately (will involve several api calls)
	TODO: nicer hour assigment"""

	dates = pd.date_range(start_end_dates[0],start_end_dates[1])
	years, months, days, hours = set(), set(), set(), []
	# extract years from time
	for date in dates:
		years.add(str(date.year))
		months.add(gtc_functions.pad_number_with_zeros(date.month))
		days.add(gtc_functions.pad_number_with_zeros(date.day))

	# generate hour strings
	for i in range(24):
		hours.append(f'{i:02d}:00')

	years, months, days = list(years), list(months), list(days)

	time_info = {"year": years,
				"month": months[0],
				"day": days,
				"hours": hours}

	return time_info


def fetch_era5_data(
	weather_keys: list[str],
	start_end_dates: list[tuple[pd.Timestamp]],
	areas: list[tuple[float]],
	download_dest_dir: str,
	format: str = 'grib'
):
	"""Generate API call, download files, merge xarrays, save as new pkl file.
	
	Parameters
	----------
	weather_keys : list[str]
		list of weather parameter short names to be included in the call
	start_end_dates : list[tuple[pd.Timestamp]]
		list of start and end date/times for each event
	area : list[tuple[float]]
		list of max/min lat/lon values in format [north, west, south, east]
	download_dest_dir : str
		path to download destination
	format : str = 'grib'
		format of data file to be downloaded
	"""
	# initialise client
	c = cdsapi.Client()

	weather_params = return_parameter_strings(weather_keys)
	for i, dates in enumerate(start_end_dates):
		# create new folder for downloads
		destination_path = gtc_functions.get_path(download_dest_dir)
		dir_name = '_'.join((
			dates[0].strftime("%d-%m-%Y"), dates[1].strftime("%d-%m-%Y")
			))
		dir_path = gtc_functions.create_dir_if_absent(destination_path, dir_name)
		
		time_info_dict = generate_times_from_start_end(dates)

		# TODO: add in nice handling for non-existent parameters
		for param in weather_params:
			# generate api call info TODO: put into function
			api_call_dict = generate_api_dict(param, time_info_dict, areas[i], format)
			file_name = f'{param}.{format}'
			dest = '/'.join((dir_path, file_name))
			# make api call 
			# TODO: is there a nice way to overwrite files of same name, provided they are
			# different? e.g. different area	
			try:
				c.retrieve(
					'reanalysis-era5-land',
					api_call_dict,
					dest
				)
			# if error in fetching, limit the parameter
			except:
				print(f'{param} not found in {dates}. Skipping fetching, moving on.')
			
		# load in all files in folder
		file_paths = '/'.join((dir_path, f'*.{format}'))

		xa_dict = {}
		for file_path in tqdm.tqdm(glob.glob(file_paths)):
			# get name of file
			file_name = file_path.split('/')[-1]
			# read into xarray
			xa_dict[file_name] = xr.load_dataset(file_path, engine="cfgrib")
			
		# merge TODO: apparently conflicting values of 'step'. Unsure why.
		out = xr.merge([array for array in xa_dict.values()], compat='override')
		# save as new file
		nc_file_name = '.'.join((dir_name, 'nc'))
		save_file_path = '/'.join((destination_path, nc_file_name))
		out.to_netcdf(path=save_file_path)
		print(f'{nc_file_name} saved successfully')

		# TODO: maybe delete folder and all files – not for now
	

In [19]:
weather_keys = ['t0', 't1', 't2', 'tp', 's0', 's1', 's2', 's3', 's4', 'e0', 'e1', 'e2', 'run0', 'run1', 'run2', 'w0', 'w1', 'sp']

fetch_era5_data(
	weather_keys = weather_keys, 
	start_end_dates = start_end_dates,
	areas = areas,
	download_dest_dir = '/Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/EFs/weather_data/ecmwf/api'
	)

2023-03-12 22:28:00,381 INFO Welcome to the CDS
2023-03-12 22:28:00,383 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-12 22:28:00,504 INFO Request is queued
2023-03-12 22:28:01,553 INFO Request is running
2023-03-12 22:28:03,100 INFO Request is completed
2023-03-12 22:28:03,102 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data9/adaptor.mars.internal-1678660080.7611442-1099-6-d993ec0d-b9d5-46c0-8f97-95ef2fbd45b5.grib to /Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/EFs/weather_data/ecmwf/api/14-09-2018_15-09-2018/2m_dewpoint_temperature.grib (34.2K)
2023-03-12 22:28:03,384 INFO Download rate 122.7K/s
2023-03-12 22:28:03,475 INFO Welcome to the CDS
2023-03-12 22:28:03,481 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-12 22:28:03,550 INFO Req

14-09-2018_15-09-2018.nc saved successfully


2023-03-12 22:29:02,461 INFO Request is running
2023-03-12 22:29:06,307 INFO Request is completed
2023-03-12 22:29:06,308 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data8/adaptor.mars.internal-1678660141.6217196-30739-16-370d066a-e849-451b-9410-adf791e5b8f0.grib to /Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/EFs/weather_data/ecmwf/api/26-08-2017_29-08-2017/2m_dewpoint_temperature.grib (83.9K)
2023-03-12 22:29:06,432 INFO Download rate 681.6K/s
2023-03-12 22:29:06,520 INFO Welcome to the CDS
2023-03-12 22:29:06,520 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-12 22:29:06,594 INFO Request is queued
2023-03-12 22:29:07,644 INFO Request is running
2023-03-12 22:29:11,686 INFO Request is completed
2023-03-12 22:29:11,687 INFO Downloading https://download-0021.copernicus-climate.eu/cache-com

26-08-2017_29-08-2017.nc saved successfully


2023-03-12 22:30:28,251 INFO Request is running
2023-03-12 22:30:29,800 INFO Request is completed
2023-03-12 22:30:29,802 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data3/adaptor.mars.internal-1678660227.484916-21971-1-70115c50-da8a-4ba6-9ffc-fe0efc2b45ee.grib to /Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/EFs/weather_data/ecmwf/api/04-10-2016_05-10-2016/2m_dewpoint_temperature.grib (9.1K)
2023-03-12 22:30:30,054 INFO Download rate 36.4K/s
2023-03-12 22:30:30,165 INFO Welcome to the CDS
2023-03-12 22:30:30,167 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-12 22:30:30,229 INFO Request is queued
2023-03-12 22:30:31,286 INFO Request is running
2023-03-12 22:30:32,995 INFO Request is completed
2023-03-12 22:30:32,996 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute

04-10-2016_05-10-2016.nc saved successfully


2023-03-12 22:31:31,374 INFO Request is running
2023-03-12 22:31:32,919 INFO Request is completed
2023-03-12 22:31:32,920 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data3/adaptor.mars.internal-1678660290.5955687-330-6-6953aa20-b9cc-4bb5-a5f2-2f6a0ab39f2a.grib to /Users/orlandotimmerman/Library/CloudStorage/GoogleDrive-rt582@cam.ac.uk/.shortcut-targets-by-id/132Xl9yWOGKPM7ybLH0oa9c3dJGYrXkjC/datasets/EFs/weather_data/ecmwf/api/10-10-2018_11-10-2018/2m_dewpoint_temperature.grib (36.6K)
2023-03-12 22:31:33,016 INFO Download rate 382.2K/s
2023-03-12 22:31:33,094 INFO Welcome to the CDS
2023-03-12 22:31:33,095 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-12 22:31:33,165 INFO Request is queued
2023-03-12 22:31:34,217 INFO Request is running
2023-03-12 22:31:35,783 INFO Request is completed
2023-03-12 22:31:35,784 INFO Downloading https://download-0017.copernicus-climate.eu/cache-comput

10-10-2018_11-10-2018.nc saved successfully


In [ ]:
for n in df_final['name'].unique()[:]:
	print(n)
	print(df_final[df_final['name'] == n].isna().sum())

In [ ]:
# ### VPTREE EXAMPLE (not used)

# # np.random.seed(42)
# # test_points = np.random.rand(100, 2)
# # test_points = np.array([[40,110],[40,-109.9],[34,90]])
# test_points = coords[:1000]
# query_point = np.array([-108,39.925])

# my_tree = vptree.VPTree(test_points, geoddist)

# [distance, closest_p] = my_tree.get_nearest_neighbor(query_point)
# print(distance, closest_p)

# fig,ax = plt.subplots(1)
# ax.scatter(x=[el[0] for el in test_points], y=[el[1] for el in test_points], label='data')
# ax.scatter(x=query_point[0], y=query_point[1], label='query')
# ax.scatter(x=closest_p[0], y=closest_p[1], color='k', marker='x', s=100, label='closest')
# ax.set_xlabel('latitude')
# ax.set_ylabel('longitude')
# ax.legend()
# ax.set_aspect('auto')

# Visualisation of weather data

In [ ]:
u10[0][0].plot(x='longitude', y='latitude', cmap='coolwarm', vmin=-20, vmax=20, figsize=(6,4))

In [ ]:
len(u10.step)

In [ ]:
num_side = 2
fig,axs = plt.subplots(num_side, num_side)
axs = axs.ravel()

plt.rcParams.update({'font.size': 6})

for i in range(num_side**2):
	# iterate over day
	for d in u10.time:
		# iterate over step
		for s in u10.step:
			u10_data = u10.sel(time=d,step=s)
			u10_data.plot(ax=axs[i], x='longitude', y='latitude', cmap='coolwarm', vmin=-20, vmax=20)
			
			# u10[i][0].plot(ax=axs[i], x='longitude', y='latitude', cmap='coolwarm', vmin=-20, vmax=20)


In [ ]:
for i in range(5):
    var_data[i,:,:].plot(x='longitude', y='latitude', figsize=(6,4))
    plt.show()
    plt.close()

In [ ]:
# Get a handle on the figure and the axes
fig, ax = plt.subplots(figsize=(12,6))

var_data = ds['u10']


# Plot the initial frame. 
cax = var_data[0,:,:].plot(
    add_colorbar=True,
    cmap='coolwarm',
    vmin=-40, vmax=40,
    cbar_kwargs={
        'extend':'neither'
    }
)

# Next we need to create a function that updates the values for the colormesh, as well as the title.
def animate(frame):
    cax.set_array(var_data[frame,:,:].values.flatten())
    ax.set_title("Time = " + str(var_data.coords['time'].values[frame])[:13])

# Finally, we use the animation module to create the animation.
ani = animation.FuncAnimation(
    fig,             # figure
    animate,         # name of the function above
    frames=40,       # Could also be iterable or list
    interval=200     # ms between frames
)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs

var_data = ds['u10']

# Create a figure and axis using cartopy
fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Define a function to update the plot for each time step
def update_plot(i):
    # Clear the axis for the new plot
    ax.clear()

    # Plot the variable data for the current time step
    var_data.isel(time=i).plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)

    # Add a title and annotation for the current time step
    ax.set_title('Variable Data at Time: {}'.format(var_data.time.values[i]), fontsize=14)
    ax.annotate('Time: {}'.format(var_data.time.values[i]), xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12)

    # Set the axis extent and gridlines
    ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
    ax.gridlines(linestyle='--', draw_labels=True)

# Create an animation object
ani = animation.FuncAnimation(fig, update_plot, frames=len(var_data.time), repeat=True)

# Save the animation as an mp4 file
ani.save('your_animation.mp4', writer='ffmpeg')
